<a href="https://colab.research.google.com/github/GUDDUemsec/Implementation-with-Faiss-IndexIVFPQ-HNSW/blob/main/Similarity_Search_by_IVFPQ_%2B_HNSW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import AutoTokenizer, TFAutoModel
import pandas as pd
import numpy as nv

In [4]:
file_data=pd.read_csv('/content/medium_Data.csv.xls')
file_data.shape
file_data

,Titles,Authors,Dates,Links,Contents
0,Please Consider the Racial Impact of Your Hall...,Savala Nolan,Oct 17,https://momentum.medium.com/i-beg-you-consider...,"Oh, it’s that time of year again! So here is a..."
1,That Time We Burned Down Players’ Houses in Ul...,Tim Cotten,Oct 3,https://blog.cotten.io/that-time-we-burned-dow...,Ultima Online is celebrating its 25th annivers...
2,The Wonderful Outcome of the Ice Bucket Challenge,Jamie Cohen,Sep 30,https://newanddigital.medium.com/the-long-payo...,The water had to be ice cold and you had to be...
3,Why Should You Care About the Bar Exam?,Dan Canon,Sep 27,https://medium.com/i-taught-the-law/why-should...,You didn’t take the bar exam. That’s probably ...
4,Please Don’t Tell My Daughter She’s Beautiful,Kerala Taylor,Oct 11,https://keralataylor.medium.com/please-dont-te...,I have two beautiful children.I might be biase...
...,...,...,...,...,...
304,How I Kept My Sense of Humor Through Vision Loss,Nancy Solari,Sep 15,https://betterhumans.pub/how-i-kept-my-sense-o...,My diagnosis of retinitis pigmentosa at age 16...
305,The true scientific meaning of Mercury enterin...,Ethan Siegel,Sep 15,https://medium.com/starts-with-a-bang/the-true...,"Throughout almost all of the year, you can see..."
306,"The Feelings Monster, building a character wit...",Microsoft Design,Sep 6,https://medium.com/microsoft-design/the-feelin...,On designing for mental health and the power o...
307,Feeling Overwhelmed? You Need an MVD—That’s Mi...,Rebecca Pendleton,"May 27, 2021",https://index.medium.com/feeling-overwhelmed-y...,I’\nm a product director at a corporate financ...


In [14]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [ ]:
!pip install datasets

In [13]:
!pip install faiss-cpu


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from datasets import Dataset

file_dataset = Dataset.from_pandas(file_data)
file_dataset

Dataset({
    features: ['Titles', 'Authors', 'Dates', 'Links', 'Contents'],
    num_rows: 309
})

In [6]:
def concatenate_text(examples):
    return {
        "text": examples["Titles"]
        }


file_dataset = file_dataset.map(concatenate_text)
file_dataset

  0%|          | 0/309 [00:00<?, ?ex/s]

Dataset({
    features: ['Titles', 'Authors', 'Dates', 'Links', 'Contents', 'text'],
    num_rows: 309
})

In [7]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [15]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [16]:
embedding = get_embeddings(file_dataset["text"])
embedding.shape
embedding.numpy()


array([[-0.15904081, -0.02815436, -0.34769472, ..., -0.05652934,
        -0.25248185, -0.2042977 ],
       [-0.02089706, -0.07984344, -0.30487287, ...,  0.1659936 ,
        -0.28794888,  0.14395118],
       [ 0.19895144,  0.01414739, -0.44468015, ..., -0.15330131,
         0.18584473, -0.1487731 ],
       ...,
       [-0.07744551, -0.01752464, -0.28358772, ...,  0.07210067,
        -0.12631136, -0.14551273],
       [-0.26690954, -0.38169014, -0.28832585, ..., -0.19347376,
         0.01754411, -0.16607675],
       [ 0.07269134, -0.473799  , -0.10616846, ...,  0.03023861,
         0.00621269,  0.05653273]], dtype=float32)

In [14]:
embeddings_dataset = file_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).numpy()[0]}
)
   

  0%|          | 0/20 [00:00<?, ?ex/s]

In [15]:
embeddings_dataset.add_faiss_index(column='embeddings')

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Titles', 'Date&Time', 'Images_URL', 'Article_links', 'Article_conetent', 'text', 'embeddings'],
    num_rows: 20
})

In [35]:
question = input("Search:")
question_embedding = get_embeddings([question]).numpy()
# question_embedding.shape
question_embedding
# embeddings_dataset

Search:https://medium.com/i-taught-the-law/why-should...


array([[ 1.74297929e-01, -7.42525831e-02, -3.49798203e-01,
        -1.93209782e-01,  5.91971166e-02,  1.64664447e-01,
         5.73514044e-01, -1.72044896e-02, -1.74832106e-01,
        -4.64584157e-02,  2.65787721e-01, -3.27996910e-04,
        -1.16459444e-01, -2.47404456e-01,  1.84394166e-01,
        -1.52583361e-01, -8.91584605e-02, -1.12524323e-01,
         1.09764338e-01, -1.41406432e-02,  1.59024239e-01,
        -5.12523875e-02, -2.82459736e-01, -2.32064575e-02,
        -9.86819118e-02, -5.66371251e-03, -1.94411367e-01,
         2.54693776e-01, -2.38133520e-01,  1.57080531e-01,
         2.44731218e-01,  4.50671524e-01, -1.92898080e-01,
         2.72307456e-01, -9.96271774e-05, -1.05062366e-01,
         1.69029325e-01, -4.90277112e-02, -5.71030416e-02,
         1.50283977e-01, -1.26317814e-01, -2.05716610e-01,
        -4.29711610e-01, -6.32224679e-02, -3.43595833e-01,
         2.03450784e-01,  2.63794828e-02, -9.14791673e-02,
         1.20999768e-01,  8.08043480e-02,  3.32165718e-0

In [17]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)
   

In [18]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
for _, data in samples_df.iterrows():
    print("~" * 200)
    print(f"Titles: {data.text}\nArticle conetent: {data.Article_conetent}\nArticle links: {data.Article_links}")
    print("~" * 200)
    

# **Implementation with Faiss:(IndexIVFPQ + HNSW) on Scraping team data.**

In [38]:
xb=embedding.numpy()[:700]
xq=question_embedding
print(xb.shape)
xq.shape


(309, 768)


(1, 768)

In [39]:
import faiss
d = xb.shape[1]      
M = 2       
k=5
index = faiss.IndexHNSWFlat(d, M)            
index.hnsw.efConstruction = 40         
index.add(xb)                  
Dis, I = index.search(xq,k)

In [40]:
print(Dis)
print(I)

[[28.014862 28.192406 31.965117 32.191536 33.89283 ]]
[[225  80 302   3 212]]


In [23]:
index.is_trained

True

In [41]:
import faiss
d = xb.shape[1]        
M = 64        
nlist = xb.shape[0] 
nsegment = 64  
nbit = 8     
k=50

coarse_quantizer = faiss.IndexHNSWFlat(d, M)
index = faiss.IndexIVFPQ(coarse_quantizer, d, nlist, nsegment, nbit)      




In [42]:
index.is_trained

False

In [ ]:
index.train(xb)



In [54]:
index.nprobe = 30 

In [55]:
Dist, I = index.search(xq, k)

In [56]:
Dis

array([[28.014862, 28.192406, 31.965117, 32.191536, 33.89283 ]],
      dtype=float32)